In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
image_labels = []
with open("ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()
image_labels[:5]

['background', 'tench', 'goldfish', 'great white shark', 'tiger shark']

In [4]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [5]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [6]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [7]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [8]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [12]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [13]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [14]:
num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [15]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
86/86 [==============================] - 63s 661ms/step - loss: 0.8499 - acc: 0.6813
Epoch 2/5
86/86 [==============================] - 55s 634ms/step - loss: 0.4192 - acc: 0.8561
Epoch 3/5
86/86 [==============================] - 55s 636ms/step - loss: 0.3334 - acc: 0.8932
Epoch 4/5
86/86 [==============================] - 55s 639ms/step - loss: 0.2788 - acc: 0.9124
Epoch 5/5
86/86 [==============================] - 55s 645ms/step - loss: 0.2385 - acc: 0.9291


In [16]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 20s 657ms/step - loss: 0.3732 - acc: 0.8747


[0.37315401434898376, 0.8747276663780212]

In [17]:
predictions = model.predict(X_test_scaled)

29/29 [==============================] - 23s 713ms/step


In [21]:
classes = ['roses', 'daisy', 'dandelion', 'sunflower', 'tulip']

In [26]:
classes[np.argmax(predictions[222])]

'dandelion'

In [27]:
classes[y_test[222]]

'dandelion'

In [28]:
IMAGE_SHAPE

(224, 224)

In [29]:
tulip = Image.open("tulip1.jfif").resize(IMAGE_SHAPE)

In [31]:
tulip = np.array(tulip)/255
tulip.shape

(224, 224, 3)

In [33]:
result_tulip = model.predict(tulip[np.newaxis, ...])

1/1 [==============================] - 0s 201ms/step


In [36]:
classes[np.argmax(result_tulip)]

'tulip'

In [37]:
def test_image():
    img_name = str(input("Enter name of image: "))
    img_open = tulip = Image.open(img_name).resize(IMAGE_SHAPE)
    img_resized = np.array(img_open)/255
    predicted_result = model.predict(img_resized[np.newaxis, ...])
    print("The predicted result is: ", classes[np.argmax(predicted_result)])

In [45]:
test_image()

Enter name of image: green_snake.jpg
1/1 [==============================] - 0s 48ms/step
The predicted result is:  roses


In [48]:
def test_image1():
    img_name = str(input("Enter name of image: "))
    img_open = tulip = Image.open(img_name).resize(IMAGE_SHAPE)
    img_resized = np.array(img_open)/255
    predicted_result = classifier.predict(img_resized[np.newaxis, ...])
    print("The predicted result is: ", image_labels[np.argmax(predicted_result)])

In [52]:
test_image1()

Enter name of image: poopi.jpg
1/1 [==============================] - 0s 46ms/step
The predicted result is:  kelpie
